change rsi_avg color 
- https://code.luasoftware.com/tutorials/algo-trading/mplfinance-change-color-depending-on-value-is-positive-or-negative/

---

# RSI study of  winning stocks 

---


In [1]:
# This allows multiple outputs from a single jupyter notebook cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np

In [3]:
import matplotlib.pyplot as plt

In [4]:
plt.rcParams.update({'grid.linestyle' : 'dotted' })

In [5]:
import mplfinance as mpf
mpf.__version__
# mpf.plot(df,volume=True,type='candle')


'0.12.8b10'

In [6]:
FLAG_DEBUG = True #  False # 

RSI_PERIOD, RSI_AVG, RSI_BAND_WIDTH = 100, 25, 0.6
EMA_FAST, EMA_SLOW, EMA_LONG = 15, 50, 150
EMA_FAST_SCALE = 1.4  # EMA10 band half-width factor
EMA_SLOW_SCALE = 2.0 
MA_VOL = 20
SPAN, OVERLAP = 200, 15
PANID_PRICE, PANID_VOL, PANID_RSI, PANID_SIGNAL = 0, 3, 2, 1
PANEL_RATIOS = (8, 1, 8, 1)
FIGURE_WIDTH, FIGURE_HEIGHT =  17, 13
YELLOW = '#F5D928'
LIGHT_BLACK = '#8F8E83'

def _ta_RSI(df, n=14):
    # https://github.com/wgong/mplfinance/blob/master/examples/rsi.py
    diff = df.w_p.diff().values
    gains = diff
    losses = -diff
    with np.errstate(invalid='ignore'):
        gains[(gains<0)|np.isnan(gains)] = 0.0
        losses[(losses<=0)|np.isnan(losses)] = 1e-10 # we don't want divide by zero/NaN
    m = (n-1) / n
    ni = 1 / n
    g = gains[n] = np.nanmean(gains[:n])
    l = losses[n] = np.nanmean(losses[:n])
    gains[:n] = losses[:n] = np.nan
    for i,v in enumerate(gains[n:],n):
        g = gains[i] = ni*v + m*g
    for i,v in enumerate(losses[n:],n):
        l = losses[i] = ni*v + m*l
    rs = gains / losses
    return 50 - (100/(1+rs))

def load_stooq_data(ticker):
    df = pd.read_csv(f'data/{ticker}.us.txt',index_col=2,parse_dates=True)
    df.reset_index(inplace=True)
    df.drop(columns=['<TICKER>', '<PER>', '<TIME>','<OPENINT>'], axis=1, inplace=True)
    df.rename(columns={'<DATE>' : "Date", '<OPEN>': "Open", '<HIGH>':"High", 
                   '<LOW>':"Low", '<CLOSE>':"Close", '<VOL>':"Volume"} ,inplace=True)
    df.set_index("Date", inplace=True)
    return df



def calculate_ta(df,rsi_period=RSI_PERIOD, avg_period=RSI_AVG, band_width=RSI_BAND_WIDTH):
    
    df["w_p"] = 0.25*(2*df["Close"] + df["High"] + df["Low"])
    df["ema_fast"] = df.w_p.ewm(span=EMA_FAST).mean()
    df["ema_slow"] = df.w_p.ewm(span=EMA_SLOW).mean()
    df["ema_long"] = df.w_p.ewm(span=EMA_LONG).mean()

    # range
    hl_mean_fast = (df.High - df.Low).ewm(span=int(EMA_FAST/2)).mean()
    df["ema_fast_u"] =  df.ema_fast + 0.5*hl_mean_fast * EMA_FAST_SCALE
    df["ema_fast_d"] =  df.ema_fast - 0.5*hl_mean_fast * EMA_FAST_SCALE

    hl_mean_slow = (df.High - df.Low).ewm(span=int(EMA_SLOW/2)).mean()
    df["ema_slow_u"] =  df.ema_slow + 0.5*hl_mean_slow * EMA_SLOW_SCALE
    df["ema_slow_d"] =  df.ema_slow - 0.5*hl_mean_slow * EMA_SLOW_SCALE

    # trim volume to avoid exponential form
    df['Volume'] = df['Volume'] / 1000000
    df["vol_avg"] = df.Volume.ewm(span=MA_VOL).mean()

    df["rsi"] = _ta_RSI(df,n=rsi_period)
    df["rsi_avg"] = df.rsi.ewm(span=avg_period).mean()
    df["rsi_u"] = df["rsi_avg"] + band_width 
    df["rsi_d"] = df['rsi_avg'] - band_width 
    df["rsi_signal"] = df["rsi"] - df["rsi_avg"]

    return df

def _title_xy(ticker):
    # position title manually
    return {"title": f"{ticker.upper()}",  "x": 0.75, "y": 0.95}

def _color_rsi_avg(v):
    if v > 0: return '#DCF7E5'
    elif v < 0: return '#F6D5F7'
    return YELLOW

def _color_signal(v):
    if v > 0: return 'g'
    elif v < 0: return 'r'
    return YELLOW

In [7]:
def gen_chart(ticker, df, num=0, panid_price=PANID_PRICE, panid_vol=PANID_VOL,
              panid_rsi=PANID_RSI, panid_signal=PANID_SIGNAL, image_path="images"):
    """
    Generate chart using Mplfinance
    """
    ema_fast_u_plot = mpf.make_addplot(df["ema_fast_u"], panel=panid_price, color=LIGHT_BLACK, linestyle="solid")
    ema_fast_d_plot = mpf.make_addplot(df["ema_fast_d"], panel=panid_price, color=LIGHT_BLACK, linestyle="solid")
    ema_slow_plot = mpf.make_addplot(df["ema_slow"], panel=panid_price, color='b', width=2, linestyle="solid")
    ema_long_plot = mpf.make_addplot(df["ema_long"], panel=panid_price, width=2, color='k')  # magenta
    
    rsi_min, rsi_max = np.nanmin(df["rsi"]), np.nanmax(df["rsi"])
    rsi_avg_plot = mpf.make_addplot(df["rsi_avg"], panel=panid_rsi, type="bar", 
                            color=[_color_rsi_avg(v)  for v in df["rsi_avg"]], 
                            ylim=(rsi_min,rsi_max))
    rsi_avg_plot2 = mpf.make_addplot(df["rsi_avg"], panel=panid_rsi, color="b", width=1, linestyle="solid",
                            ylim=(rsi_min,rsi_max))
    rsi_plot = mpf.make_addplot(df["rsi"], panel=panid_rsi, color='r', width=1, #title=f"{ticker}-RSI",
                            fill_between=dict(y1=df["rsi_d"].values,y2=df["rsi_u"].values,alpha=0.15,color='b'),
                            ylim=(rsi_min,rsi_max))
    signal_plot = mpf.make_addplot(df["rsi_signal"], panel=panid_signal, type="bar", 
                            color=[_color_signal(v) for v in df["rsi_signal"]], 
                            ylim=(-1,1))                            

    vol_avg_plot = mpf.make_addplot(df["vol_avg"], panel=panid_vol, color='k')

    plots = [
            # panel-0
            ema_fast_u_plot, ema_fast_d_plot, ema_slow_plot, ema_long_plot # ema_slow_u_plot, ema_slow_d_plot, 
            # panel-1
            , rsi_avg_plot, rsi_plot, rsi_avg_plot2  # , rsi_u_plot, rsi_d_plot 
            # panel-2
            , vol_avg_plot
            # panel-3
            , signal_plot
        ]
    file_img = f"{image_path}/{ticker}-{str(num).zfill(2)}.png"
    print(file_img)
    mpf.plot(df, type='candle', 
            style='yahoo', 
            fill_between=dict(y1=df["ema_fast_d"].values,y2=df["ema_fast_u"].values,alpha=0.15,color='b'),
            panel_ratios=PANEL_RATIOS,
            addplot=plots, 
            title=_title_xy(ticker),
            volume=True, volume_panel=panid_vol, 
            ylabel="", ylabel_lower='',
            xrotation=0,
            datetime_format='%Y-%m-%d',
            savefig=file_img,
            figsize=(FIGURE_WIDTH,FIGURE_HEIGHT),
            tight_layout=True,
            show_nontrading=True
        )
    return {"ticker": ticker, "file_img": file_img}

In [8]:
for ticker in ["aapl", "adbe", "amd", "amzn", "googl", "intc"]:

    # test
    if FLAG_DEBUG and ticker != "aapl" : continue

    df = load_stooq_data(ticker)
    df = calculate_ta(df)
    
    LEN = df.shape[0]

    num = 0
    intervals = []
    for ix_stop in range(LEN, RSI_PERIOD, -SPAN):
        ix_start = ix_stop - SPAN -int(OVERLAP/2)
        intervals.append((ix_start if ix_start > 0 else 0, ix_stop+int(OVERLAP/2)))    

    for ix_start,ix_stop in intervals[::-1]:
        num += 1

        # test
        if FLAG_DEBUG and num > 2:  break
        
        df_1 = df.iloc[ix_start : ix_stop]
        gen_chart(ticker, df_1, num=num)

images/aapl-01.png
images/aapl-02.png


In [9]:
df_1.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'w_p', 'ema_fast', 'ema_slow',
       'ema_long', 'ema_fast_u', 'ema_fast_d', 'ema_slow_u', 'ema_slow_d',
       'vol_avg', 'rsi', 'rsi_avg', 'rsi_u', 'rsi_d', 'rsi_signal'],
      dtype='object')

In [10]:
df_1.dtypes

Open          float64
High          float64
Low           float64
Close         float64
Volume        float64
w_p           float64
ema_fast      float64
ema_slow      float64
ema_long      float64
ema_fast_u    float64
ema_fast_d    float64
ema_slow_u    float64
ema_slow_d    float64
vol_avg       float64
rsi           float64
rsi_avg       float64
rsi_u         float64
rsi_d         float64
rsi_signal    float64
dtype: object

In [11]:
df_1.index

DatetimeIndex(['1985-05-15', '1985-05-16', '1985-05-17', '1985-05-20',
               '1985-05-21', '1985-05-22', '1985-05-23', '1985-05-24',
               '1985-05-28', '1985-05-29',
               ...
               '1986-03-06', '1986-03-07', '1986-03-10', '1986-03-11',
               '1986-03-12', '1986-03-13', '1986-03-14', '1986-03-17',
               '1986-03-18', '1986-03-19'],
              dtype='datetime64[ns]', name='Date', length=214, freq=None)

## resample